---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [4]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [6]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    fin = open("university_towns.txt")
    lines = fin.readlines()
    current_state = ''
    row_liste = []
    liste = []
    accu = 1
    for line in lines:
        if re.search(r'\[edit\]', line):
            current_state = re.sub(r'\[edit\]', '', line).strip()
            accu+=1
        else:
            accu+=1
            #print(accu, current_state, line)
            line = re.sub(r'\(.*', '', line)  #remove parenthesis (some lines don't have the closing parenthesis)
            line = line.strip()  #line is 1 region (1 per line)
            #print(current_state, line)
            row_liste = [current_state]+[line]
            liste.append(row_liste)
    #print(liste, len(liste))
    uni_towns_df = pd.DataFrame(liste, columns=["State", "RegionName"])
    pd.set_option('display.max_rows', len(uni_towns_df))
    #print(uni_towns_df)
    pd.reset_option('display.max_rows')
    uni_towns_df.reset_index()
    return uni_towns_df

In [7]:
U_towns = get_list_of_university_towns()

In [8]:
#Creating GDP df
GDP = pd.read_excel("gdplev.xls",
                #header=6, 
                skiprows=219,
                names=['Quarter', 'GDP in billions of chained 2009 dollars'],
                index_col=None,
                parse_cols=[4,6],
                skip_footer=0)

In [9]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    #Running loop to find 2 consecutive declining quarters
    gdp_q1=''
    gdp_q2=''
    for row in GDP.itertuples():  #fast way to iterate through df (better: np arrays)
        if row.Index==0:
            gdp_q1=row._2
        elif row.Index==1:
            gdp_q2=row._2
        else:
            if gdp_q2<gdp_q1 and row._2<gdp_q2:
                return GDP.loc[GDP['GDP in billions of chained 2009 dollars']==gdp_q2, 'Quarter'].item()
            else:
                gdp_q1=gdp_q2
                gdp_q2=row._2
    return 'No recession found'

In [10]:
recession_start = get_recession_start()

In [11]:
recession_start

'2008q3'

In [12]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    #Running loop to find 2 consecutive up quarters
    gdp_q1=''
    gdp_q2=''
    for row in GDP.itertuples():  #fast way to iterate through df (better: np arrays)
        if row.Index==0:
            gdp_q1=row._2
        elif row.Index==1:
            gdp_q2=row._2
        else:
            if row.Quarter>recession_start and gdp_q2>gdp_q1 and row._2>gdp_q2:  #adding condition 'after recession'
                return GDP.loc[GDP['GDP in billions of chained 2009 dollars']==row._2, 'Quarter'].item()
            else:
                gdp_q1=gdp_q2
                gdp_q2=row._2
    return 'No end in sight'

In [13]:
recession_end = get_recession_end()

In [14]:
recession_end

'2009q4'

In [15]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    min = ''
    for row in GDP.itertuples():  #fast way to iterate through df (better: np arrays)
        if row.Quarter==recession_start:
            min=row._2
        else:
            if row.Quarter>recession_start and row.Quarter<recession_end and row._2<min:
                min=row._2
    return GDP.loc[GDP['GDP in billions of chained 2009 dollars']==min, 'Quarter'].item()

In [16]:
recession_bottom = get_recession_bottom()

In [17]:
recession_bottom

'2009q2'

In [18]:
import re
    
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].

    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.

    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    columns=[1,2]+[i for i in range(51,251)]
    Housing = pd.read_csv("City_Zhvi_AllHomes.csv",
                header=0, 
                #skiprows=219,
                #names=['Quarter', 'GDP in billions of chained 2009 dollars'],
                #index_col=[1,0], #works but changing states values first
                usecols=columns,
                #skip_footer=0
                         )
    Housing.State=Housing.State.apply(lambda x: states.get(x, "Missing State"))   #replacing 2 letters w state name
    Housing.set_index([Housing.State, Housing.RegionName], drop=True, inplace=True) #creating multi-index
    Housing.drop(['State', 'RegionName'], axis=1, inplace=True)
    Housing['2016-09']=np.NaN

    for i in range(16):
        s=str(2000+i)
        Housing[s+'q1'] =  (1/3)*(Housing[s+'-01']+Housing[s+'-02']+Housing[s+'-03'])
        Housing[s+'q2'] =  (1/3)*(Housing[s+'-04']+Housing[s+'-05']+Housing[s+'-06'])
        Housing[s+'q3'] =  (1/3)*(Housing[s+'-07']+Housing[s+'-08']+Housing[s+'-09'])
        Housing[s+'q4'] =  (1/3)*(Housing[s+'-10']+Housing[s+'-11']+Housing[s+'-12'])
    #2016
    s='2016'
    Housing[s+'q1'] =  (1/3)*(Housing[s+'-01']+Housing[s+'-02']+Housing[s+'-03'])
    Housing[s+'q2'] =  (1/3)*(Housing[s+'-04']+Housing[s+'-05']+Housing[s+'-06'])
    Housing[s+'q3'] =  (1/3)*(Housing[s+'-07']+Housing[s+'-08']+Housing[s+'-09'])


    for col in Housing:
        if re.search(r'-', col)!=None:
            Housing.drop(col, axis=1, inplace=True)

    return Housing

In [19]:
Housing = convert_housing_data_to_quarters()

In [20]:
Housing.shape

(10730, 67)

In [21]:
Housing.head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,NaN
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,NaN
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,NaN
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,NaN
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,NaN


In [22]:
import re
import pandas as pd
q5 = convert_housing_data_to_quarters()
res = 'Output type test: '
if type(q5) == pd.DataFrame:
    res += 'Passed\n'
    pattern = re.compile("\d\d\d\dq[1-4]")
    res += 'Columns type test: '
    if type(q5.columns) == pd.indexes.base.Index:
        res += 'Passed'
        res += '\nColumns names test: '
        if q5.columns.map(pattern.match).astype(bool).all():
            res += 'Passed'
        else:
            res += 'Failed'
            res += "\n Do the column names look like ['2000q1','2000q2'....]"
            res += '\n PS. column names are case sensitive'
    else:
        res += 'Failed'
        res += "\n Did you remember to change the column names back "
        res += "to strings in the format e.g. Index(['2000q1', '2000q2',...],dtype='object')"
    res += "\nState name test: "
    if set(q5.index.get_level_values(0)).issubset(set(states.values())):
        res += 'Passed'
    else:
        res += 'Failed'
        res += "\n Did you map the abbreviated state names to the full names"
        res += " using the precoded states dictionary"
    res += "\nShape test: "
    if q5.shape == (10730, 67):
        res += 'Passed'
    else:
        res += 'Failed'

    # generate all column names
    columns = set(pd.date_range(
        start="2000", end="2017", freq='Q').map(
        lambda x: "{:}q{:}".format(x.year, x.quarter))[:-1])
    res += "\nColumn Names test: "
    if set(q5.columns) == columns:
        res += 'Passed'
    else:
        res += 'Failed'
        res += "\nmissing columns" + str(columns - set(q5.columns))
else:
    res += "Failed\n Output type is {}, a pandas.DataFrame is expected".format(
        type(q5))
print(res)

Output type test: Passed
Columns type test: Passed
Columns names test: Passed
State name test: Passed
Shape test: Passed
Column Names test: Passed


In [23]:
from scipy import stats

def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    Housing['Change']=Housing[recession_start].div(Housing[recession_bottom])
    U_index = U_towns.set_index(['State', 'RegionName']).index
    U_towns_Housing = Housing[Housing.index.isin(U_index)]
    Reg_towns_Housing = Housing[~Housing.index.isin(U_index)]
    res = stats.ttest_ind(U_towns_Housing['Change'], Reg_towns_Housing['Change'], nan_policy='omit')
    
    return (True, res.pvalue, 'university town')

In [24]:
run_ttest()

(True, 0.0059928062139885064, 'university town')